In [7]:
import numpy as np
import scipy as sp
import math
import Universality_check as Uc
import Gate_helper

### States

In [2]:
zero  = np.array([[1], [0], [0], [0], [0], [0]],dtype=complex)
one   = np.array([[0], [1], [0], [0], [0], [0]],dtype=complex)
two   = np.array([[0], [0], [1], [0], [0], [0]],dtype=complex)
three = np.array([[0], [0], [0], [1], [0], [0]],dtype=complex)
four  = np.array([[0], [0], [0], [0], [1], [0]],dtype=complex)
five  = np.array([[0], [0], [0], [0], [0], [1]],dtype=complex)

### Permutations

In [3]:
SN = - zero@zero.T - one@one.T + five@two.T \
     +four@three.T+three@four.T+ two@five.T

SS =  -zero@zero.T - one@one.T + five@three.T \
     +four@two.T+two@four.T+ three@five.T

SE =  -two@two.T - three@three.T + five@one.T \
     +four@zero.T+zero@four.T+ one@five.T

SW = - two@two.T - three@three.T + five@zero.T \
     +four@one.T+one@four.T+ zero@five.T

D1 = zero@three.T + three@zero.T +one@two.T + two@one.T \
     -four@four.T - five@five.T

D2 = zero@two.T + two@zero.T +one@three.T + three@one.T \
     -four@four.T - five@five.T

rot_CW = zero@three.T + three@one.T + one@two.T + two@zero.T + four@five.T + five@four.T

rot_CCW = rot_CW.T

flip_V = zero@zero.T + one@one.T + three@two.T + two@three.T + four@five.T + five@four.T

flip_H = zero@one.T + one@zero.T + three@three.T + two@two.T + four@five.T + five@four.T

Id = zero@zero.T + one@one.T + two@two.T + \
     three@three.T + four@four.T + five@five.T

### Gate Set (S)

In [11]:
Perms = [SN,SS,SE,SN]#D1,D2,flip_H,flip_V,rot_CW,rot_CCW]
SQRT_SWAP = [(Id + 1j*P)/np.sqrt(2) for P in Perms]
Quarter_Swap = [np.cos(np.pi/8)*Id + np.sin(np.pi/8)*1j*P for P in Perms]
S = SQRT_SWAP

### Step 1 of the algorithm,

For our gate set to be universal we need the

In [12]:
Uc.check_Center(S)

False

### Step 2 of the algorithm

This code determines the span of our gate set is infinite or finite. It does this by attempting to find an element that can be reached from our gate set that is in a ball of radius 1 that is not in the center of the group. Not if this check pases, but the previous check fails that the span of $S$ is infinite, but not all of $SU(6)$, meaning it is not universal.  

In [13]:
# using a much smaller value for N that is required to be thourough since i want this to actually run 
N_SU6 = 100  #36398100 # upper bound for N
Uc.check_Finite(S,N_SU6,10)

l =  0 current size =  4
l =  1 current size =  12
Infinite [[ 0.5-0.5j  0. +0.j   0. +0.j   0. +0.j   0.5+0.5j  0. +0.j ]
 [ 0. +0.j   0.5-0.5j  0. +0.j   0. +0.j   0. +0.j   0.5+0.5j]
 [ 0. +0.j  -0.5+0.j   0.5-0.5j  0. +0.j   0. +0.j   0. +0.5j]
 [-0.5+0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.5j  0. +0.j ]
 [ 0. +0.5j  0. +0.j   0. +0.j   0.5+0.5j  0.5+0.j   0. +0.j ]
 [ 0. +0.j   0. +0.5j  0.5+0.5j  0. +0.j   0. +0.j   0.5+0.j ]]


[False, -1]

## Or all together we can check for universality


In [14]:
Uc.check_universal(S)

False